### 1. Import libraries and define base class/ functions

In [8]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import ImageChops
from skimage import io, color
import ast
import os
from PIL import Image
import glob
import ast

In [9]:
def convert_tif_into_png(directory,path):
    '''
    Function to convert .tif into .png files.
    Input: directory/path of .tif images
    Output: .png images
    '''
    c=1
    for filename in os.listdir(directory):

        if filename.endswith(".tif"):
            name=os.path.splitext(filename)[0]
            im = Image.open(path+filename)
            name=name+'-raw.png'
            rgb_im = im.convert('RGB')
            rgb_im.save(name)
            c+=1
            print(os.path.join(directory, filename))
            continue
        else:
            continue
            
# my_directory= '/Users/TramNguyen/Downloads/IFN705/CUBIC-Annotated/Compare_label/Masktif'
# my_path='Compare_label/Masktif/'
#convert_tif_into_png(my_directory,my_path)

In [10]:
def convert_into_bw(image):
    '''
    Function to convert masks into binary 
    with the confidence threshold 99%.
    
    Input: annotated image in RGB
    Output: binary mask
    '''
    img_grey=color.rgb2gray(image)
    thresh = 0.99
    img_bw = cv2.threshold(img_grey, thresh, 1, cv2.THRESH_BINARY)[1]
    return img_bw

In [11]:
# Program to count cells in boolean 2D matrix
# using DFS -  Reference: https://www.geeksforgeeks.org/find-number-of-islands/

class Graph:
    '''
    This class is to implement Depth First Search on pixels that are detected to be the neuron.
    Then connected pixels are grouped into one neuron.
    After that, count the number of neurons in the image.
    '''
 
    def __init__(self, row, col, g):
        '''
        Initialize objects
        '''
        self.ROW = row
        self.COL = col
        self.graph = g
 

    def isSafe(self, i, j, visited,list_i=[],list_j=[]):
        '''
        This function is to check if a given cell (row, col) can be included in DFS
        '''
        # row number is in range, column number
        # is in range and value is 1
        # and not yet visited
        return (i >= 0 and i < self.ROW and
                j >= 0 and j < self.COL and
                not visited[i][j] and self.graph[i][j])
             
        
    def DFS(self, i, j, visited, maxi, maxj,mini,minj):
        '''
        A utility function to do DFS for a 2D boolean matrix. 
        It only considers the 8 neighbours as adjacent vertices
        '''
        
        # These arrays are used to get row and
        # column numbers of 8 neighbours
        # of a given cell
        rowNbr = [-1, -1, -1,  0, 0,  1, 1, 1];
        colNbr = [-1,  0,  1, -1, 1, -1, 0, 1];
         
        # Mark this cell as visited
        visited[i][j] = True

        currentmaxi = maxi
        currentmaxj = maxj
        currentmini = mini
        currentminj = minj

        # Recur for all connected neighbours
        for k in range(8):
            nexti = i + rowNbr[k]
            nextj = j + colNbr[k]
            if self.isSafe(nexti, nextj, visited):
                nextmaxi,nextmaxj,nextmini,nextminj = self.DFS(nexti, nextj, visited, max(currentmaxi, nexti),max(currentmaxj, nextj), min(currentmini, nexti),min(currentminj, nextj))
                currentmaxi = max(currentmaxi,nextmaxi)
                currentmaxj = max(currentmaxj,nextmaxj)
                currentmini = min(currentmini,nextmini)
                currentminj = min(currentminj,nextminj)
    
        return currentmaxi, currentmaxj,currentmini,currentminj
        
 
    
    def countCells(self):
        '''
        This function is to return the count of cells in a given boolean 2D matrix
        '''
        # Make a bool array to mark visited cells.
        # Initially all cells are unvisited
        visited = [[False for j in range(self.COL)]for i in range(self.ROW)]
 
        # Initialize count as 0 and travese
        # through the all cells of
        # given matrix
        count = 0
        centroids=[]

        for i in range(self.ROW):
            for j in range(self.COL):
                # If a cell with value 1 is not visited yet,
                # then new island found
                if visited[i][j] == False and self.graph[i][j] == 1:
                    # Visit all cells in this island
                    # and increment island count
                    
                    maxi,maxj,mini,minj=self.DFS(i, j, visited,i,j,i,j)
                    cent_x=round((maxi+mini)/2)
                    cent_y=round((maxj+minj)/2)
                    count += 1
                    centroids.append((cent_x,cent_y))
        return count, centroids

In [12]:
# stable marriage algorithms to compare human and machine annotations
# the function is encapsulated  for the convenience of use 

def stable_pairs(img_1,img_2):  
    '''
    Input: 
        img_1 is mask labelled by hand
        img_2 is mask by the model
    Output: return the matched pairs, and the accuracy metrics
    '''
    a_img=cv2.imread(img_1)
    b_img=cv2.imread(img_2)
    img_raw=convert_into_bw(a_img)
    img_pre=convert_into_bw(b_img)
    graph_raw=Graph(len(img_raw), len(img_raw[0]), img_raw)
    graph_pre=Graph(len(img_pre), len(img_pre[0]), img_pre)
    A=graph_raw.countCells()[1]
    B=graph_pre.countCells()[1]
    len_human_label=((graph_raw.countCells()[0]))
    len_machine_label=((graph_pre.countCells()[0]))
    lenA= len_human_label
    lenB= len_machine_label
  
    
    def make_NxN_arrays(array1,array2):    
        '''
        This function is to make the length of 2 arrays equal to each other by adding pseudo points.
        Input: 2 given arrays
        Output: 2 new arrays whose lengths are equal.
        '''
        if len(array1)>len(array2):
            i = len(array1)-len(array2)
            for x in range (0,i):
                array2.insert(len(array1),(10000+x,10000+x))
                x=x+1
        elif len(array1)<len(array2):
            i = len(array2)-len(array1)
            for x in range (0,i):
                array1.insert(len(array2),(10000+x,10000+x))
                x=x+1
        return array1, array2


    def convert_into_string(list_tuple):
        '''
        This function is to convert list in to string
        '''
        res = []
        for i in list_tuple:
            res.append(str(i))
        return res

    
    def distance_a_to_b(a, b):
        '''
        This function is to calculate the distance between 2 points, applying Euclidean distance
        '''
        return ((a[0] - b[0])**2 + (a[1] - b[1])**2) ** 0.5


    def match(a):
        '''
        This function is to apply stable marriage problem algorithm to match pairs in 2 arrays
        '''
        for b in A_map[a]:
            paired_values = [pair for pair in stable_pairs if b in pair]
            #print("PV: ", paired_values)
            if (len(paired_values) == 0):
                stable_pairs.append([a, b])
                free_a.remove(a)
                break

            elif (len(paired_values) > 0):
                current_a = B_map[b].index(paired_values[0][0])
                next_a = B_map[b].index(a)
                if (current_a >= next_a):
                    free_a.remove(a)
                    free_a.append(paired_values[0][0])
                    paired_values[0][0] = a
                    break   
        return stable_pairs


    def check_distance(pairs, threshold):
        '''
        This function is to return the list of pairs which satisfy the distance limit.
        Use threshold parameter to set the distance. For example, threshold= 8 means that 8 pixels away. 
        '''
        list_of_pairs=[]
        for pair in pairs:
            dist=((pair[0][0] - pair[1][0])**2 + (pair[0][1] - pair[1][1])**2) ** 0.5
            if dist<= threshold:
                list_of_pairs.append(pair)
        return list_of_pairs

                
    def convert_into_tuple(string_array):
    #convert tuple into string
        tuple_ = []
        for index, tuple in enumerate(string_array):
            element_one = tuple[0]
            element_one=ast.literal_eval(element_one)
            element_two = tuple[1]
            element_two=ast.literal_eval(element_two)
            tuple_.append([element_one,element_two])
        return tuple_


    A,B=make_NxN_arrays(A,B)
    N = len(A)
    A_map = {}
    B_map = {}
    for i in range(0, N):
        a = str(A[i])
        b = str(B[i])
        A_map[a] = convert_into_string(sorted(B, key=lambda e: distance_a_to_b(e, A[i])))
        B_map[b] = convert_into_string(sorted(A, key=lambda e: distance_a_to_b(e, B[i])))

    stable_pairs = []
    free_a = []

    for a in A_map.keys():
        free_a.append(a)

    while(len(free_a) > 0):
        for a in free_a:
            sp=match(a)
    pairs=convert_into_tuple(sp)
    thres=8
    final_pairs=check_distance(pairs, thres)
    count=len(final_pairs)
    match_percentage=(len(final_pairs)/lenA)*100
    human_label=A
    machine_label=B
   
    TP=count
    FP=abs(count-len_machine_label)
    FN=abs(count-len_human_label)
    precision=TP/(TP+FP)*100
    recall=TP/(TP+FN)*100
    
    
    return final_pairs,count, match_percentage,  human_label,  machine_label, len_human_label, len_machine_label, precision, recall

### Implement on the data

In [13]:
raw_masks_dir='Compare_label/raw_mask_png/'
predicted_masks_dir='Compare_label/predicted_masks/'
file_ids = [x.rsplit('.',1)[0] for x in os.listdir(predicted_masks_dir)]
file_ids.remove("")
raw_masks=[(os.path.join(raw_masks_dir, x)) for x in [s + '-raw.png' for s in file_ids]]
predicted_masks=[(os.path.join(predicted_masks_dir, x)) for x in [s + '.png' for s in file_ids]]

In [14]:
raw_masks


['Compare_label/raw_mask_png/6-mask-raw.png',
 'Compare_label/raw_mask_png/7-mask-raw.png',
 'Compare_label/raw_mask_png/20-mask-raw.png',
 'Compare_label/raw_mask_png/14-mask-raw.png',
 'Compare_label/raw_mask_png/15-mask-raw.png',
 'Compare_label/raw_mask_png/1-mask-raw.png',
 'Compare_label/raw_mask_png/19-mask-raw.png',
 'Compare_label/raw_mask_png/18-mask-raw.png',
 'Compare_label/raw_mask_png/13-mask-raw.png',
 'Compare_label/raw_mask_png/12-mask-raw.png',
 'Compare_label/raw_mask_png/17-mask-raw.png',
 'Compare_label/raw_mask_png/16-mask-raw.png',
 'Compare_label/raw_mask_png/4-mask-raw.png',
 'Compare_label/raw_mask_png/10-mask-raw.png',
 'Compare_label/raw_mask_png/11-mask-raw.png',
 'Compare_label/raw_mask_png/8-mask-raw.png',
 'Compare_label/raw_mask_png/9-mask-raw.png',
 'Compare_label/raw_mask_png/2-mask-raw.png',
 'Compare_label/raw_mask_png/3-mask-raw.png']

In [15]:
predicted_masks

['Compare_label/predicted_masks/6-mask.png',
 'Compare_label/predicted_masks/7-mask.png',
 'Compare_label/predicted_masks/20-mask.png',
 'Compare_label/predicted_masks/14-mask.png',
 'Compare_label/predicted_masks/15-mask.png',
 'Compare_label/predicted_masks/1-mask.png',
 'Compare_label/predicted_masks/19-mask.png',
 'Compare_label/predicted_masks/18-mask.png',
 'Compare_label/predicted_masks/13-mask.png',
 'Compare_label/predicted_masks/12-mask.png',
 'Compare_label/predicted_masks/17-mask.png',
 'Compare_label/predicted_masks/16-mask.png',
 'Compare_label/predicted_masks/4-mask.png',
 'Compare_label/predicted_masks/10-mask.png',
 'Compare_label/predicted_masks/11-mask.png',
 'Compare_label/predicted_masks/8-mask.png',
 'Compare_label/predicted_masks/9-mask.png',
 'Compare_label/predicted_masks/2-mask.png',
 'Compare_label/predicted_masks/3-mask.png']

In [16]:
accuracy_list=[]
precision_list=[]
recall_list=[]

for a,b in zip(raw_masks,predicted_masks):
    final_pairs=[]
    count=0
    match_percentage=0
    print('************************************************')
    print('COMPARE', a,' AND', b)
    final_pairs,count, match_percentage, human_label,machine_label,len_human_label,len_machine_label,precision,recall=stable_pairs(a,b)

    print ("The number of matched pair is ",count," which accounts for ", match_percentage," percent of accuracy." )
    print ("Precision: ",precision, ". Recall: ",recall)
    print ("Matched pairs are: ")
    print(final_pairs)
    accuracy_list.append(match_percentage)
    precision_list.append(precision)
    recall_list.append(recall)
    
    

************************************************
COMPARE Compare_label/raw_mask_png/6-mask-raw.png  AND Compare_label/predicted_masks/6-mask.png
The number of matched pair is  48  which accounts for  82.75862068965517  percent of accuracy.
Precision:  51.06382978723404 . Recall:  82.75862068965517
Matched pairs are: 
[[(181, 388), (181, 388)], [(262, 316), (262, 316)], [(279, 337), (278, 338)], [(317, 279), (318, 280)], [(326, 274), (326, 274)], [(341, 313), (341, 312)], [(344, 282), (344, 282)], [(360, 264), (360, 264)], [(378, 648), (378, 648)], [(411, 685), (411, 686)], [(486, 725), (486, 726)], [(493, 724), (492, 724)], [(502, 394), (501, 394)], [(510, 734), (510, 734)], [(522, 380), (522, 382)], [(534, 368), (534, 368)], [(538, 363), (539, 363)], [(536, 379), (538, 380)], [(543, 360), (543, 360)], [(547, 363), (547, 363)], [(548, 372), (548, 372)], [(552, 384), (552, 384)], [(553, 362), (553, 362)], [(556, 730), (556, 730)], [(557, 383), (557, 383)], [(561, 354), (560, 354)], [(56

The number of matched pair is  9  which accounts for  69.23076923076923  percent of accuracy.
Precision:  75.0 . Recall:  69.23076923076923
Matched pairs are: 
[[(214, 558), (214, 558)], [(246, 416), (246, 416)], [(270, 388), (270, 388)], [(275, 382), (275, 382)], [(492, 258), (492, 259)], [(509, 781), (510, 780)], [(256, 720), (256, 720)], [(314, 363), (313, 363)], [(503, 831), (503, 830)]]
************************************************
COMPARE Compare_label/raw_mask_png/18-mask-raw.png  AND Compare_label/predicted_masks/18-mask.png
The number of matched pair is  33  which accounts for  56.896551724137936  percent of accuracy.
Precision:  89.1891891891892 . Recall:  56.896551724137936
Matched pairs are: 
[[(206, 718), (206, 718)], [(172, 331), (172, 331)], [(214, 721), (214, 721)], [(224, 312), (224, 312)], [(232, 310), (232, 310)], [(267, 714), (267, 714)], [(296, 320), (296, 320)], [(328, 816), (328, 816)], [(318, 266), (319, 266)], [(370, 318), (370, 319)], [(399, 837), (399, 837

The number of matched pair is  3  which accounts for  75.0  percent of accuracy.
Precision:  75.0 . Recall:  75.0
Matched pairs are: 
[[(296, 290), (296, 290)], [(461, 741), (460, 741)], [(562, 786), (562, 786)]]
************************************************
COMPARE Compare_label/raw_mask_png/9-mask-raw.png  AND Compare_label/predicted_masks/9-mask.png
The number of matched pair is  2  which accounts for  100.0  percent of accuracy.
Precision:  66.66666666666666 . Recall:  100.0
Matched pairs are: 
[[(626, 738), (626, 738)], [(663, 722), (662, 722)]]
************************************************
COMPARE Compare_label/raw_mask_png/2-mask-raw.png  AND Compare_label/predicted_masks/2-mask.png
The number of matched pair is  25  which accounts for  54.347826086956516  percent of accuracy.
Precision:  73.52941176470588 . Recall:  54.347826086956516
Matched pairs are: 
[[(318, 491), (318, 490)], [(329, 572), (329, 572)], [(350, 459), (350, 458)], [(351, 620), (351, 620)], [(353, 444), (

In [188]:
#accuracy_list
#avg_acc=sum(accuracy_list)/len(accuracy_list)
#avg_acc

In [190]:
recall_list

[82.75862068965517,
 86.11111111111111,
 67.94871794871796,
 70.0,
 72.22222222222221,
 57.34767025089605,
 69.23076923076923,
 56.896551724137936,
 73.80952380952381,
 100.0,
 67.21311475409836,
 80.0,
 77.08333333333334,
 61.42857142857143,
 68.11594202898551,
 75.0,
 100.0,
 54.347826086956516,
 75.0]

In [191]:
avg_recall=sum(recall_list)/len(recall_list)
avg_recall

73.39547234836729

In [193]:
precision_list
precision_avg=sum(precision_list)/len(precision_list)
precision_avg

65.53932980138386

### Test on one image

In [176]:
final_pairs,count, match_percentage, human_label, machine_label,len_human_label,len_machine_label,precision,recall=stable_pairs('Compare_label/raw_mask_png/9-mask-raw.png','Compare_label/predicted_masks/9-mask.png')


In [177]:
#human_label YES vs machine_label YES: true positive
#human_label YES vs machine_label NO: false positive
#human_label NO vs machine_label YES: false negative
#Precision=TP/(TP+FP)
#Recall=TP/(TP+FN)
print("Accuracy:", match_percentage )
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 100.0
Precision: 100.0
Recall: 66.66666666666666


In [178]:
human_label

[(626, 738), (663, 722), (10000, 10000)]

In [179]:
machine_label

[(626, 738), (650, 241), (662, 722)]